# Adv Games Regression
<a id="top"></a>

Treating games as the sum of each team's players stats.
0. [Helper Methods](#helpers)
1. [Score Differential Regressor](#score-diff)
    * [SVD regression](#score-diff-lr), [RFT regression](#score-diff-logr)
    * Only take raw game scores and betting lines
2. [Past Win% Differential Regressor](#winper)
    * [SVD regression](#winper-lr), [RFT regression](#winper-logr)
    * Take past win% over 10, 25, 50, 75, 100 games for each team
3. [Box Score Regressor](#boxscore)
    * Takes in overall box score for either team 

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.decomposition import PCA

In [2]:
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:5}'.format

## Helper Methods
[to top](#top)
<a id="helpers">

In [3]:
def SVGRegression(X_input, y_input):
    n_folds = 5
    kf = KFold(n_folds, shuffle=True)

    # Create a pipeline that first standardizes the data, then applies PCA, and finally trains the model
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components="mle")),  # k is the number of principal components
        ('regressor', SVR())
    ])

    # Perform cross-validation
    cv_scores = cross_val_score(pipeline, X_input, np.ravel(y_input), cv=kf, scoring='neg_mean_squared_error')

    # Calculate average MSE
    average_mse = -np.mean(cv_scores)
    print("cv:", cv_scores)
    print("Average MSE:", average_mse)
    print("Average RMSE:", np.sqrt(average_mse))

    return pipeline

def SVGRegressionFit(X_input, y_input):

    X_train, X_test, y_train, y_test = train_test_split(X_input, y_input, test_size=.2)

    # Create a pipeline that first standardizes the data, then applies PCA, and finally trains the model
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components="mle")),  # k is the number of principal components
        ('regressor', SVR())
    ])

    pipeline.fit(X_train, np.ravel(y_train))
    # y_pred = pipeline.predict(X_test)

    # # The coefficients
    # # The mean squared error
    # print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
    # # The coefficient of determination: 1 is perfect prediction
    # print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

    return pipeline

## Rolling Regressors
[to top](#top)

-  [linear regression](#winper-lr)
-  [logistic regression](#winper-logr)
<a id="winper">

In [52]:
df = pd.read_csv("data_proc/games_rolling_master.csv")
df.dropna(inplace=True)
# display(df)
game_info = ['TEAM_ID_T1', 'TEAM_ID_T2', 'DATE', 'away_prob', 'away_line', 'home_prob', 'home_line', 'away_spread', 'away_total'] 
intervals = ['10g_', '25g_', '50g_', '75g_', '100g']
columns = game_info + [c for c in df.columns.to_list() if c not in game_info and c[:4] in intervals]
print(columns)
print(df.shape[0])

['TEAM_ID_T1', 'TEAM_ID_T2', 'DATE', 'away_prob', 'away_line', 'home_prob', 'home_line', 'away_spread', 'away_total', '10g_FGM_T1', '10g_FGM_T2', '10g_FGA_T1', '10g_FGA_T2', '10g_FG3M_T1', '10g_FG3M_T2', '10g_FG3A_T1', '10g_FG3A_T2', '10g_FTM_T1', '10g_FTM_T2', '10g_FTA_T1', '10g_FTA_T2', '10g_OREB_T1', '10g_OREB_T2', '10g_DREB_T1', '10g_DREB_T2', '10g_REB_T1', '10g_REB_T2', '10g_AST_T1', '10g_AST_T2', '10g_STL_T1', '10g_STL_T2', '10g_BLK_T1', '10g_BLK_T2', '10g_TO_T1', '10g_TO_T2', '10g_PF_T1', '10g_PF_T2', '10g_PTS_T1', '10g_PTS_T2', '10g_PLUS_MINUS_T1', '10g_PLUS_MINUS_T2', '10g_FG_PCT_T1', '10g_FG_PCT_T2', '10g_FG3_PCT_T1', '10g_FG3_PCT_T2', '10g_FT_PCT_T1', '10g_FT_PCT_T2', '10g_SCORE_DIFF_T1', '10g_SCORE_DIFF_T2', '10g_WON_T1', '10g_WON_T2', '25g_FGM_T1', '25g_FGM_T2', '25g_FGA_T1', '25g_FGA_T2', '25g_FG3M_T1', '25g_FG3M_T2', '25g_FG3A_T1', '25g_FG3A_T2', '25g_FTM_T1', '25g_FTM_T2', '25g_FTA_T1', '25g_FTA_T2', '25g_OREB_T1', '25g_OREB_T2', '25g_DREB_T1', '25g_DREB_T2', '25g_REB_T

In [63]:
df

,GAME_ID,SBR_GAME_ID,DATE,TEAM_ID_T1,FGM_T1,FGA_T1,FG3M_T1,FG3A_T1,FTM_T1,FTA_T1,OREB_T1,DREB_T1,REB_T1,AST_T1,STL_T1,BLK_T1,TO_T1,PF_T1,PTS_T1,PLUS_MINUS_T1,FG_PCT_T1,FG3_PCT_T1,FT_PCT_T1,SCORE_DIFF_T1,WON_T1,TEAM_ID_T2,FGM_T2,FGA_T2,FG3M_T2,FG3A_T2,FTM_T2,FTA_T2,OREB_T2,DREB_T2,REB_T2,AST_T2,STL_T2,BLK_T2,TO_T2,PF_T2,PTS_T2,PLUS_MINUS_T2,FG_PCT_T2,FG3_PCT_T2,FT_PCT_T2,SCORE_DIFF_T2,WON_T2,away_line,home_line,away_prob,home_prob,away_spread,away_total,10g_FGM_T1,10g_FGM_T2,10g_FGA_T1,10g_FGA_T2,10g_FG3M_T1,10g_FG3M_T2,10g_FG3A_T1,10g_FG3A_T2,10g_FTM_T1,10g_FTM_T2,10g_FTA_T1,10g_FTA_T2,10g_OREB_T1,10g_OREB_T2,10g_DREB_T1,10g_DREB_T2,10g_REB_T1,10g_REB_T2,10g_AST_T1,10g_AST_T2,10g_STL_T1,10g_STL_T2,10g_BLK_T1,10g_BLK_T2,10g_TO_T1,10g_TO_T2,10g_PF_T1,10g_PF_T2,10g_PTS_T1,10g_PTS_T2,10g_PLUS_MINUS_T1,10g_PLUS_MINUS_T2,10g_FG_PCT_T1,10g_FG_PCT_T2,10g_FG3_PCT_T1,10g_FG3_PCT_T2,10g_FT_PCT_T1,10g_FT_PCT_T2,10g_SCORE_DIFF_T1,10g_SCORE_DIFF_T2,10g_WON_T1,10g_WON_T2,25g_FGM_T1,25g_FGM_T2,25g_FGA_T1,25g_FGA_T2,25g_FG3M_T1,25g_FG3M_T2,25g_FG3A_T1,25g_FG3A_T2,25g_FTM_T1,25g_FTM_T2,25g_FTA_T1,25g_FTA_T2,25g_OREB_T1,25g_OREB_T2,25g_DREB_T1,25g_DREB_T2,25g_REB_T1,25g_REB_T2,25g_AST_T1,25g_AST_T2,25g_STL_T1,25g_STL_T2,25g_BLK_T1,25g_BLK_T2,25g_TO_T1,25g_TO_T2,25g_PF_T1,25g_PF_T2,25g_PTS_T1,25g_PTS_T2,...,50g_FGM_T2,50g_FGA_T1,50g_FGA_T2,50g_FG3M_T1,50g_FG3M_T2,50g_FG3A_T1,50g_FG3A_T2,50g_FTM_T1,50g_FTM_T2,50g_FTA_T1,50g_FTA_T2,50g_OREB_T1,50g_OREB_T2,50g_DREB_T1,50g_DREB_T2,50g_REB_T1,50g_REB_T2,50g_AST_T1,50g_AST_T2,50g_STL_T1,50g_STL_T2,50g_BLK_T1,50g_BLK_T2,50g_TO_T1,50g_TO_T2,50g_PF_T1,50g_PF_T2,50g_PTS_T1,50g_PTS_T2,50g_PLUS_MINUS_T1,50g_PLUS_MINUS_T2,50g_FG_PCT_T1,50g_FG_PCT_T2,50g_FG3_PCT_T1,50g_FG3_PCT_T2,50g_FT_PCT_T1,50g_FT_PCT_T2,50g_SCORE_DIFF_T1,50g_SCORE_DIFF_T2,50g_WON_T1,50g_WON_T2,75g_FGM_T1,75g_FGM_T2,75g_FGA_T1,75g_FGA_T2,75g_FG3M_T1,75g_FG3M_T2,75g_FG3A_T1,75g_FG3A_T2,75g_FTM_T1,75g_FTM_T2,75g_FTA_T1,75g_FTA_T2,75g_OREB_T1,75g_OREB_T2,75g_DREB_T1,75g_DREB_T2,75g_REB_T1,75g_REB_T2,75g_AST_T1,75g_AST_T2,75g_STL_T1,75g_STL_T2,75g_BLK_T1,75g_BLK_T2,75g_TO_T1,75g_TO_T2,75g_PF_T1,75g_PF_T2,75g_PTS_T1,75g_PTS_T2,75g_PLUS_MINUS_T1,75g_PLUS_MINUS_T2,75g_FG_PCT_T1,75g_FG_PCT_T2,75g_FG3_PCT_T1,75g_FG3_PCT_T2,75g_FT_PCT_T1,75g_FT_PCT_T2,75g_SCORE_DIFF_T1,75g_SCORE_DIFF_T2,75g_WON_T1,75g_WON_T2,100g_FGM_T1,100g_FGM_T2,100g_FGA_T1,100g_FGA_T2,100g_FG3M_T1,100g_FG3M_T2,100g_FG3A_T1,100g_FG3A_T2,100g_FTM_T1,100g_FTM_T2,100g_FTA_T1,100g_FTA_T2,100g_OREB_T1,100g_OREB_T2,100g_DREB_T1,100g_DREB_T2,100g_REB_T1,100g_REB_T2,100g_AST_T1,100g_AST_T2,100g_STL_T1,100g_STL_T2,100g_BLK_T1,100g_BLK_T2,100g_TO_T1,100g_TO_T2,100g_PF_T1,100g_PF_T2,100g_PTS_T1,100g_PTS_T2,100g_PLUS_MINUS_T1,100g_PLUS_MINUS_T2,100g_FG_PCT_T1,100g_FG_PCT_T2,100g_FG3_PCT_T1,100g_FG3_PCT_T2,100g_FT_PCT_T1,100g_FT_PCT_T2,100g_SCORE_DIFF_T1,100g_SCORE_DIFF_T2,100g_WON_T1,100g_WON_T2
1401,21500895.0,20160301_nba_Atlanta_Golden State,20160301.0,1610612737.0,37.0,80.0,12.0,33.0,19.0,24.0,7.0,35.0,42.0,25.0,9.0,5.0,17.0,17.0,105.0,-20.0,0.4625,0.3636363636363636,0.7916666666666666,-4.0,0.0,1610612744.0,41.0,103.0,12.0,36.0,15.0,17.0,18.0,33.0,51.0,28.0,11.0,3.0,11.0,22.0,109.0,20.0,0.3980582524271844,0.3333333333333333,0.8823529411764706,4.0,1.0,195.0,-235.0,0.3389830508474577,0.7014925373134329,5.5,214.0,40.4,43.2,88.6,88.7,10.8,13.2,29.9,33.3,12.9,17.5,17.1,23.4,9.6,9.2,36.5,36.0,46.1,45.2,27.3,28.7,9.8,8.7,6.5,4.7,15.5,15.1,18.2,21.4,104.5,117.1,31.0,33.0,0.4578881681099283,0.4891526101377616,0.3776933456904267,0.4014877673226005,0.7495,0.7489642005920116,6.2,6.6,0.5,0.9,40.28,42.6,86.84,87.16,10.2,12.72,29.24,30.44,14.08,16.76,18.16,22.04,8.8,9.12,34.4,36.48,43.2,45.6,27.04,29.36,9.96,7.84,6.08,5.12,15.2,14.52,20.04,20.72,104.84,114.68,...,42.4,84.64,86.66,9.66,12.9,27.66,30.48,15.44,17.54,19.42,23.18,8.44,9.94,33.08,36.0,41.52,45.94,26.36,29.06,9.72,8.32,5.8,5.88,14.64,15.12,19.24,20.54,103.74,115.24,20.6,53.4,0.4655660290016027,0.4905916260689076,0.3533073597267006,0.4223305863

In [16]:
SVGRegression(df[columns].values, df["SCORE_DIFF_T1"].values)


cv: [-161.33991349 -181.62280012 -165.55217305 -189.33365199 -165.5745361 ]
Average MSE: 172.68461494997678
Average RMSE: 13.140951828158293


Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components='mle')),
                ('regressor', SVR())])

In [17]:
away_spread_reg = SVGRegressionFit(df.iloc[:-1000][columns].values, df[:-1000]["SCORE_DIFF_T1"].values)
                
# SVGRegression(df[['TEAM1_ID', 'TEAM2_ID', 'DATE', 'TEAM1_10wp', 'TEAM1_25wp', 'TEAM1_50wp', 'TEAM1_75wp', 'TEAM1_100wp', 'TEAM2_10wp', 'TEAM2_25wp', 'TEAM2_50wp', 'TEAM2_75wp', 'TEAM2_100wp']],df[["SCORE_DIFF_T1"]])

# SVGRegression(df.drop(columns=['SCORE_DIFF_T1', 'SCORE_DIFF_T2', '']), df[["SCORE_DIFF_T1"]])
# fig = px.scatter(df, x="away_spread", y="SCORE_DIFF", hover_data=["SBR_GAME_ID"], trendline="ols")
# fig.show()


In [51]:
buffer = 6

def calcOutcomes(i):
    away_spread_reg = SVGRegressionFit(df.iloc[:-i][columns].values, df[:-i]["SCORE_DIFF_T1"].values)
    
    # create result dataframe
    result = pd.DataFrame()    
    result['SBR_GAME_ID'] = df.iloc[-i:]['SBR_GAME_ID'] # game id
    result['pred_away_spread'] = -away_spread_reg.predict(df.iloc[-i:][columns].values) # predicted away spread
    result['bet_away_spread'] = df.iloc[-i:]['away_spread']
    result['diff_away_spread'] =  result['bet_away_spread'] - result['pred_away_spread']
    result['actual_away_spread'] = df.iloc[-i:]['SCORE_DIFF_T1']
    result['away_line'] = df.iloc[-i:]['away_line']
    result['home_line'] = df.iloc[-i:]['home_line']
    result['WON_T1'] = df.iloc[-i:]['WON_T1']
    result['WON_T2'] = df.iloc[-i:]['WON_T2']


    ### Spread Betting
    print("=== SPREAD BETTING ===")

    # spread betting
    result['spread_bet_decision'] = np.abs(result['diff_away_spread']) > buffer
    result.loc[result['diff_away_spread'] > 0, 'spread_bet_on'] = 1
    result.loc[result['diff_away_spread'] < 0, 'spread_bet_on'] = 2

    result.loc[(result['spread_bet_on'] == 1) | (result['spread_bet_on'] == 2), 'spread_profit'] = 100 * (100 / 110)

    result.loc[(result['spread_bet_on'] == 1) & (result['actual_away_spread'] < -result['bet_away_spread']), 'spread_profit'] = -100 
    result.loc[(result['spread_bet_on'] == 2) & (result['actual_away_spread'] > result['bet_away_spread']), 'spread_profit'] = -100
    result.loc[(result['actual_away_spread'] == result['bet_away_spread']), 'spread_profit'] = 0

    result['spread_bet_on'] = result['spread_bet_on'] * result['spread_bet_decision']
    result['spread_profit'] = result['spread_profit'] * result['spread_bet_decision']

    # display(result)

    totalGames = result.shape[0]
    totalBets = np.count_nonzero(result["spread_bet_on"])
    betRate = np.count_nonzero(result["spread_bet_on"]) / result.shape[0] 
    avgProfitPerGame = np.average(result['spread_profit'])
    avgProfitPerBet = np.average(result['spread_profit'].loc[result['spread_bet_on'] != 0])

    # displays

    # control pure random
    print("Mathematical Random Control")
    # print("    (assume 50% bets converges)")
    expPPB = (100 * (100 / 110) - 100) / 2
    print(f"exp profit per bet                      :   {expPPB}")
    print(f"exp profit per game                     :   {1 * expPPB}5")
    print(f"exp profit per game w/ model bet rate   :   {betRate * expPPB}")
    print()

    print("w/ Model")
    print(f"total games                             :    {totalGames}")
    print(f"total bets                              :    {totalBets}")
    print(f"betRate                                 :    {betRate}")
    print(f"avg profit per game                     :    {avgProfitPerGame}")
    print(f"avg profit per bet                      :    {avgProfitPerBet}")


    # moneyline

    '''
    
    result['bet_decision'] = np.abs(result['diff_away_spread']) > buffer
    result.loc[result['diff_away_spread'] > 0, 'bet_on'] = 1
    result.loc[result['diff_away_spread'] < 0, 'bet_on'] = 2
    result.loc[(result['bet_on'] == 1) & (result['away_line'] > 0), 'profit'] = 100 * (100 / 110)
    result.loc[(result['bet_on'] == 1) & (result['away_line'] < 0), 'profit'] = 100 * (100 / 110)

    result.loc[(result['bet_on'] == 1) & (result['actual_away_spread'] < -result['bet_away_spread'] ), 'profit'] = -100 # 

    result.loc[(result['bet_on'] == 2) & (result['home_line'] > 0), 'profit'] = result['home_line'] 
    result.loc[(result['bet_on'] == 2) & (result['home_line'] < 0), 'profit'] = 100 * (100 / - result['home_line']) + 100
    result.loc[(result['bet_on'] == 2) & (result['actual_away_spread'] > result['bet_away_spread'] ), 'profit'] = -100

    result['bet_on'] = result['bet_on'] * result['bet_decision']
    result['profit'] = result['profit'] * result['bet_decision']
    '''


    return avgProfitPerGame, avgProfitPerBet, totalBets, totalGames, betRate 
    

# for i in range(100):
#     testIndex(i)
calcOutcomes(1500)

=== SPREAD BETTING ===
Mathematical Random Control
exp profit per bet                      :   -4.545454545454547
exp profit per game                     :   -4.5454545454545475
exp profit per game w/ model bet rate   :   -0.290909090909091

w/ Model
total games                             :    1500
total bets                              :    96
betRate                                 :    0.064
avg profit per game                     :    -1.5636363636363637
avg profit per bet                      :    -24.431818181818183


(-1.5636363636363637, -24.431818181818183, 96, 1500, 0.064)

In [8]:
df = pd.read_csv("data_proc/games_rolling_master.csv")
df

,GAME_ID,SBR_GAME_ID,DATE,TEAM_ID_T1,FGM_T1,FGA_T1,FG3M_T1,FG3A_T1,FTM_T1,FTA_T1,OREB_T1,DREB_T1,REB_T1,AST_T1,STL_T1,BLK_T1,TO_T1,PF_T1,PTS_T1,PLUS_MINUS_T1,FG_PCT_T1,FG3_PCT_T1,FT_PCT_T1,SCORE_DIFF_T1,WON_T1,TEAM_ID_T2,FGM_T2,FGA_T2,FG3M_T2,FG3A_T2,FTM_T2,FTA_T2,OREB_T2,DREB_T2,REB_T2,AST_T2,STL_T2,BLK_T2,TO_T2,PF_T2,PTS_T2,PLUS_MINUS_T2,FG_PCT_T2,FG3_PCT_T2,FT_PCT_T2,SCORE_DIFF_T2,WON_T2,away_line,home_line,away_prob,home_prob,away_spread,away_total,10g_FGM_T1,10g_FGM_T2,10g_FGA_T1,10g_FGA_T2,10g_FG3M_T1,10g_FG3M_T2,10g_FG3A_T1,10g_FG3A_T2,10g_FTM_T1,10g_FTM_T2,10g_FTA_T1,10g_FTA_T2,10g_OREB_T1,10g_OREB_T2,10g_DREB_T1,10g_DREB_T2,10g_REB_T1,10g_REB_T2,10g_AST_T1,10g_AST_T2,10g_STL_T1,10g_STL_T2,10g_BLK_T1,10g_BLK_T2,10g_TO_T1,10g_TO_T2,10g_PF_T1,10g_PF_T2,10g_PTS_T1,10g_PTS_T2,10g_PLUS_MINUS_T1,10g_PLUS_MINUS_T2,10g_FG_PCT_T1,10g_FG_PCT_T2,10g_FG3_PCT_T1,10g_FG3_PCT_T2,10g_FT_PCT_T1,10g_FT_PCT_T2,10g_SCORE_DIFF_T1,10g_SCORE_DIFF_T2,10g_WON_T1,10g_WON_T2,25g_FGM_T1,25g_FGM_T2,25g_FGA_T1,25g_FGA_T2,25g_FG3M_T1,25g_FG3M_T2,25g_FG3A_T1,25g_FG3A_T2,25g_FTM_T1,25g_FTM_T2,25g_FTA_T1,25g_FTA_T2,25g_OREB_T1,25g_OREB_T2,25g_DREB_T1,25g_DREB_T2,25g_REB_T1,25g_REB_T2,25g_AST_T1,25g_AST_T2,25g_STL_T1,25g_STL_T2,25g_BLK_T1,25g_BLK_T2,25g_TO_T1,25g_TO_T2,25g_PF_T1,25g_PF_T2,25g_PTS_T1,25g_PTS_T2,...,50g_FGM_T2,50g_FGA_T1,50g_FGA_T2,50g_FG3M_T1,50g_FG3M_T2,50g_FG3A_T1,50g_FG3A_T2,50g_FTM_T1,50g_FTM_T2,50g_FTA_T1,50g_FTA_T2,50g_OREB_T1,50g_OREB_T2,50g_DREB_T1,50g_DREB_T2,50g_REB_T1,50g_REB_T2,50g_AST_T1,50g_AST_T2,50g_STL_T1,50g_STL_T2,50g_BLK_T1,50g_BLK_T2,50g_TO_T1,50g_TO_T2,50g_PF_T1,50g_PF_T2,50g_PTS_T1,50g_PTS_T2,50g_PLUS_MINUS_T1,50g_PLUS_MINUS_T2,50g_FG_PCT_T1,50g_FG_PCT_T2,50g_FG3_PCT_T1,50g_FG3_PCT_T2,50g_FT_PCT_T1,50g_FT_PCT_T2,50g_SCORE_DIFF_T1,50g_SCORE_DIFF_T2,50g_WON_T1,50g_WON_T2,75g_FGM_T1,75g_FGM_T2,75g_FGA_T1,75g_FGA_T2,75g_FG3M_T1,75g_FG3M_T2,75g_FG3A_T1,75g_FG3A_T2,75g_FTM_T1,75g_FTM_T2,75g_FTA_T1,75g_FTA_T2,75g_OREB_T1,75g_OREB_T2,75g_DREB_T1,75g_DREB_T2,75g_REB_T1,75g_REB_T2,75g_AST_T1,75g_AST_T2,75g_STL_T1,75g_STL_T2,75g_BLK_T1,75g_BLK_T2,75g_TO_T1,75g_TO_T2,75g_PF_T1,75g_PF_T2,75g_PTS_T1,75g_PTS_T2,75g_PLUS_MINUS_T1,75g_PLUS_MINUS_T2,75g_FG_PCT_T1,75g_FG_PCT_T2,75g_FG3_PCT_T1,75g_FG3_PCT_T2,75g_FT_PCT_T1,75g_FT_PCT_T2,75g_SCORE_DIFF_T1,75g_SCORE_DIFF_T2,75g_WON_T1,75g_WON_T2,100g_FGM_T1,100g_FGM_T2,100g_FGA_T1,100g_FGA_T2,100g_FG3M_T1,100g_FG3M_T2,100g_FG3A_T1,100g_FG3A_T2,100g_FTM_T1,100g_FTM_T2,100g_FTA_T1,100g_FTA_T2,100g_OREB_T1,100g_OREB_T2,100g_DREB_T1,100g_DREB_T2,100g_REB_T1,100g_REB_T2,100g_AST_T1,100g_AST_T2,100g_STL_T1,100g_STL_T2,100g_BLK_T1,100g_BLK_T2,100g_TO_T1,100g_TO_T2,100g_PF_T1,100g_PF_T2,100g_PTS_T1,100g_PTS_T2,100g_PLUS_MINUS_T1,100g_PLUS_MINUS_T2,100g_FG_PCT_T1,100g_FG_PCT_T2,100g_FG3_PCT_T1,100g_FG3_PCT_T2,100g_FT_PCT_T1,100g_FT_PCT_T2,100g_SCORE_DIFF_T1,100g_SCORE_DIFF_T2,100g_WON_T1,100g_WON_T2
0,21400001.0,20141028_nba_Orlando_New Orleans,20141028.0,1610612753.0,32.0,84.0,4.0,11.0,16.0,21.0,16.0,40.0,56.0,17.0,5.0,9.0,18.0,25.0,84.0,-85.0,0.3809523809523809,0.3636363636363636,0.7619047619047619,-17.0,0.0,1610612740.0,41.0,101.0,4.0,17.0,15.0,31.0,26.0,36.0,62.0,20.0,10.0,17.0,9.0,17.0,101.0,85.0,0.4059405940594059,0.2352941176470588,0.4838709677419355,17.0,1.0,380.0,-475.0,0.2083333333333333,0.8260869565217391,9.0,191.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN